In [ ]:
#Import the libraries
import math
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

In [ ]:
itemCategories=pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
salesTrain=pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
submission=pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops=pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
test.columns.values

Tablo da bulunan satur ve sütun sayısı

In [ ]:
print("itemCategories:", itemCategories.shape)
print("items:",items.shape)
print("salesTrain",salesTrain.shape)
print("submission",submission.shape)
print("shops",shops.shape)
print("test",test.shape)

In [ ]:
itemCategories.info()
items.info()
salesTrain.info()
submission.info()
shops.info()
test.info()

In [ ]:
itemCategories.head(5)

In [ ]:
items.head(5)

In [ ]:
df = salesTrain.groupby(by=['date'],sort=True,as_index=False).count()
df

In [ ]:
submission.head(5)

In [ ]:
shops.head(5)

In [ ]:
test.head(5)

Veri setinde ki eksik değerlerin bulunması

In [ ]:
def show_missing_values(function_data):
#Veri setindeki eksik değerleri bulalım
    number_of_sample=function_data.shape[0]
    check_isnull=function_data.isnull().sum()
    
    check_isnull=check_isnull[check_isnull!=0].sort_values(ascending=False)

    if check_isnull.shape[0]==0:
        print("Data set does not contain missing data")
    else:
        print("Data set contains missing data")

In [ ]:
show_missing_values(itemCategories)
show_missing_values(items)
show_missing_values(salesTrain)
show_missing_values(submission)
show_missing_values(shops)
show_missing_values(test)

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Tarih ve Fiyat Satışları")
plt.plot(df.date,df.item_price)
plt.xlabel('Date',fontsize=12)
plt.ylabel('Price',fontsize=12)
plt.show()

In [ ]:
df = salesTrain.groupby(by=['shop_id'],sort=True,as_index=False).count()
df

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Tarih ve Fiyat Satışları")
plt.plot(shops.shop_name,df.item_price)
plt.xlabel('Shop Name',fontsize=18)
plt.ylabel('Sum Price',fontsize=18)
plt.show()

item_cnt_day den oluşan bir dataframe oluşturuldu
DataFrame diziye dönüştürüldü
Eğitim verilerinin yüzde 80 aktarıldı

In [ ]:
data = salesTrain.filter(['item_cnt_day'])

dataset = data.head(1000)

training_data_lenght = (math.ceil( len(dataset) * .8))
training_data_lenght

Veri seti 0 ile 1 arasında ölçeklendirildi.
Sinir ağına girmeden önce verilerin ölçeklendirilmesi gerekir bu iyi bir yöntemdir.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(dataset)
scaled_data.size

Örneğin 11'ıncı satış sayısını tahmin etmek istiyoruz bunun için son 60 gün item_cnt_day değerlerini içeren bir eğitim veri kümesi oluşturuldu.

In [ ]:
train_data = scaled_data[ 0:training_data_lenght,: ]
train_data.size

In [ ]:
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60 :i,0])
    y_train.append(train_data[i,0])


x_train ve y_train numpy dizelerine dönüştürüldü.

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

 Verileri 3 boyutlu olacak şekilde yeniden şek,llendirme
 Örnek sayısı, Zaman adımı sayısı ve Özellik sayısı 

In [ ]:
 x_train = np.reshape (x_train, (x_train.shape [0], x_train.shape [1], 1))

LSTM modeli oluşturma işlemi 

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape = ( x_train.shape[1] , 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

Modelin ortalama kare hatası (MSE) kaybı işlevini kullanarak derlenmesi

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

Eğitim veri seti kullanarak modelin eğitilmesi. 
Epochs tüm veri kümesinin sinir ağı üzerinden ileri ve geri aktarıldığında ki yineleme sayısıdır. 

In [ ]:
model.fit(x_train, y_train, batch_size=1, epochs=1)

Bir tane test datası oluşturuldu.

In [ ]:
 test_data = scaled_data[training_data_lenght - 60:,:]

In [ ]:
x_test = []
y_test =  dataset[scaled_data.size : , : ] 
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])

Test verisinin numpy dizisine dönüştürülmesi

In [ ]:
x_test = np.array(x_test)

Verileri 3 boyutlu olarak oluşturuyoruz.
Örnek sayısı Zaman Adım Sayısı 
Özellik Sayısı
LSTM modeli 3 boyutlu bir veriseti bekliyor.

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

Modelden tahmin edilen verilerin alınması

In [ ]:
predictions = model.predict(x_test) 
predictions = scaler.inverse_transform(predictions)

RMSE kök ortalama kare hatasının hesaplanması 
Modelin doğruluğuna dair bir ölçüttür.
0 değeri model ile gerçek değerlerin tamamen eşleştiğini gösterir.
Değerin düşük olması modelin iyi olduğunu gösterir

In [ ]:
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse